In [1]:
import import_ipynb
from arrival_networkx import *
import numpy as np
import random as rm
from typing import List
from collections import deque
import networkx as nx
import matplotlib.pyplot as plt

importing Jupyter notebook from arrival_networkx.ipynb


In [2]:
def get_optimal_phi(number_of_nodes):
    return np.sqrt(3) / np.sqrt(2*number_of_nodes)

In [3]:
def multi_run_procedure(instance: Arrival, S_subset: List[int],w : dict):
       
    t = {s:0 for s in instance.vertices}
    
    t[0] = 1     # t[o] ← 1 /* traversal of (Y, o) */
    for v in S_subset:
        t[instance.s_0[v]] += np.ceil(w[v]/2) # /* dwi/2e traversals of (vi, seven(vi)) */
        t[instance.s_1[v]] += np.floor(w[v]/2) # /* bwi/2c traversals of (vi, sodd(vi)) */
    

    # Let scurr and snext be arrays indexed by the vertices of V \ S
    s_curr =  instance.s_0 
    s_next = instance.s_1
            
            
    waiting_set = []
    for v in instance.vertices:
        if v not in S_subset and v not in [instance.target_node, instance.sink_node]:
            waiting_set.append(v)
            
    while len(waiting_set) > 0: ## maintain all the vertices not in S 
        waiting_set_ = [ws for ws in waiting_set if t[ws]>0]
        if not waiting_set_:
            break
        
        choose = rm.choice(waiting_set_)
        tau = rm.randint(1,t[choose])
        
        t[choose] -= tau
        t[s_curr[choose]] += np.ceil(tau/2)
        t[s_next[choose]] += np.floor(tau/2)
        
        if tau & 1 : 
            temp = s_curr[choose]
            s_curr[choose] = s_next[choose]
            s_next[choose] = temp
    # print(t)      
    return t[instance.target_node], t[instance.sink_node], {v:t[v] for v in S_subset}


In [29]:
def decompose_into_layers(instance: Arrival):
    """
    Decompose the graph into layers based on the distance of the vertices to the destination nodes.
    
    Parameters:
    - graph: A NetworkX graph instance.
    - target_node: The target node (d).
    - sink_node: The sink node (d').
    
    Returns:
    - layers: A dictionary where key is the layer index and value is a list of nodes in that layer.
    - max_dist: The maximum distance (layer index) found.
    """
    graph = instance.graph    
    # Initialize the layers dictionary
    layers = {}
    # Distance dictionary with initial values set to None for each node
    dist = {node: float('inf') for node in graph.nodes()}
    
    # Define a BFS procedure to calculate distances to {target_node, sink_node}
    queue = deque([(instance.target_node, 0), (instance.sink_node, 0)])
    while queue:
        current_node, current_dist = queue.popleft()
        if dist[current_node] == float('inf'):
            dist[current_node] = current_dist
            if current_dist not in layers:
                layers[current_dist] = [current_node]
            else:
                layers[current_dist].append(current_node)
            for neighbor in graph.predecessors(current_node):
                if dist[neighbor] == float('inf'):
                    queue.append((neighbor, current_dist + 1))
    
    # Calculate max_dist
    print(dist)
    max_dist = max(dist.values())
    
    return layers, max_dist

In [30]:
def compute_phi_set(instance: Arrival, phi: float) :
    layers, max_dist = decompose_into_layers(instance) 
    print(max_dist)
    S = []
    U = layers[0]
    for i in range(1,max_dist+1):
        if len(layers[i]) < phi*len(U):
            S += layers[i]
            U = []
        U += layers[i]
    return S
    

In [31]:
def compare_w(w,w_new):
    # if w is None:
    #     return False
    for key in w:
        if w[key] != w_new[key]:
            return False
    return True

def subexponential(instance: Arrival, phi: float):
    S = compute_phi_set(instance, phi)
    print("S is ",S)
    w = {s:10 for s in S}
    t_d,t_sink, w_new = multi_run_procedure(instance, S, w)
    print(w_new)
    counter = 1
    while not compare_w(w,w_new):
        w = w_new
        t_d, t_sink, w_new = multi_run_procedure(instance, S, w)
        counter += 1
        
    print("t_d is ",t_d, "t_sink is ",t_sink, "w is ",w_new)
        
    return t_d, t_sink, w_new, counter
    
    

# Validating The Algo

In [36]:
def run_procedure(instance: Arrival):
        v = instance.start_node
        s_curr = np.copy(instance.s_0) # current switches for each node
        s_next = np.copy(instance.s_1) # next switch for each node
        counter = 0
        while counter < instance.n * 2**instance.n:
            # if counter % 10**8 == 0:
            # print(f'move {counter} :{v}')
            
            if v == -1:
                return True
            elif v == instance.target_node:
                return True
            
            w = s_curr[v]
            s_curr[v] = s_next[v]
            s_next[v] = w  
            v = w
            
            counter += 1
        
        return False

In [8]:
# ## Normal random instance
# instances = []
# def run_random_instance():
#     number_of_nodes = 30
#     phi = np.sqrt(3) / np.sqrt(2*number_of_nodes)
#     # branch_instace = get_branch_instance(number_of_nodes,split_ratio=0.5)
#     for i in range(10):
#         print("---------------------------------\n","Instance ",i+1)
#         instance = Arrival(number_of_nodes, True)
#         instances.append(instance)
        
#         out = subexponential(instance, phi)
#         if out[0] > 0 and out[1] == 0:
#             if not run_procedure(instance):
#                 return instance
#     return None
            
# instance = run_random_instance()
# instance

In [9]:
# ## BRANCH Instances 
# number_of_nodes = 30
# weird_instances = []
# non_terminal_instances = []
# instances = []
# phi = np.sqrt(3) / np.sqrt(2*number_of_nodes)
# for i in range(20):    
#     r = np.random.uniform(0.1, 0.9)
#     instance = get_branch_instance(number_of_nodes, split_ratio=r)
#     instances.append(instance)
    
#     out = subexponential(instance, phi)
#     if out[0] == 0 and out[1] == 0:
#         weird_instances.append(i)
#     elif out[0] == 0 and out[1] != 0:
#         non_terminal_instances.append(i)

In [10]:
# ### Branch without random 
# number_of_nodes = 30
# weird_instances = []
# non_terminal_instances = []
# instances = []
# phi = np.sqrt(3) / np.sqrt(2*number_of_nodes)
# for i in range(1):
#         print("---------------------------------\n","Instance ",i+1)
#         instance = get_branch_instance_without_random(number_of_nodes, split_ratio=0.4)
#         instances.append(instance)
        
#         out = subexponential(instance, phi)
#         if out[0] > 0:
#             if not run_procedure(instance):
#                 break